In [1]:
import rioxarray
import geopandas
import leafmap
import os
import datetime
from osgeo import gdal
from tqdm import tqdm


In [2]:
grib_file = "era_5_2021_02-01_01.grib"
dest_file = "data/_era_5_tiff/" + "era_5_2021_02-01_01.tiff"


In [3]:
src_ds = gdal.Open(grib_file)
bands = [] # Set up array for gdal.Translate(). 
if src_ds is not None:
    bandNum = src_ds.RasterCount # Get band count
for i in range(bandNum+1): # Update array based on band count
    if (i==0): #gdal starts band counts at 1, not 0 like the Python for loop does.
        pass
    else:
        bands.append(i)

# Open output format driver
out_form= "GTiff"

# Output to new format using gdal.Translate. See https://gdal.org/python/ for osgeo.gdal.Translate options.
dst_ds = gdal.Translate(dest_file, src_ds, format=out_form, bandList=bands)

# Properly close the datasets to flush to disk
dst_ds = None
src_ds = None

Warning 1: data/_era_5_tiff/era_5_2021_02-01_01.tiff: Metadata exceeding 32000 bytes cannot be written into GeoTIFF. Transferred to PAM instead.
